In [3]:
import os
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

def make_sequences(data, win=60):
    X, y = [], []
    for i in range(len(data) - win):
        X.append(data[i : i+win])
        y.append(data[i+win, 0])
    return np.array(X), np.array(y)

def main():
    # ←—— Set your parameters here! ——————————————————————
    ticker_symbol = "AAPL"           # e.g. "BRK-B" or "MSFT"
    start_date    = "2015-01-01"     # format YYYY-MM-DD
    end_date      = "2025-03-30"     # format YYYY-MM-DD
    # ————————————————————————————————————————————————————

    print(f"Ticker: {ticker_symbol}, Start: {start_date}, End: {end_date}")

    # 1) Fetch historical data
    df = yf.Ticker(ticker_symbol).history(start=start_date, end=end_date)
    if df.empty:
        raise RuntimeError("No data returned. Check ticker & date range.")

    df.reset_index(inplace=True)
    df.sort_values("Date", inplace=True)

    # 2) Prepare 'Close' prices and scale
    close_prices = df[['Close']].values
    scaler = MinMaxScaler((0,1))
    scaled_close = scaler.fit_transform(close_prices)

    # 3) Build sequences (60-day window)
    WINDOW = 60
    X, y = make_sequences(scaled_close, win=WINDOW)
    split = int(len(X) * 0.8)
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    # 4) Define & train LSTM
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(WINDOW, 1)),
        Dropout(0.2),
        LSTM(32),
        Dropout(0.2),
        Dense(1),
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=25, batch_size=32,
              validation_split=0.1, verbose=1)

    # 5) Forecast next business day
    last_window = scaled_close[-WINDOW:].reshape(1, WINDOW, 1)
    pred_scaled = model.predict(last_window, verbose=0)[0][0]
    pred_close  = scaler.inverse_transform([[pred_scaled]])[0][0]

    last_date = df['Date'].iloc[-1]
    next_bday = last_date + pd.tseries.offsets.BDay()

    print(f"\nPredicted CLOSE for {ticker_symbol} on {next_bday.date()}: ${pred_close:.2f}\n")

    # 6) Save to CSV
    out = pd.DataFrame({
        'Ticker': [ticker_symbol],
        'Date':   [next_bday.date()],
        'Predicted_Close': [pred_close]
    })
    path = os.path.join(os.path.dirname(__file__),
                        f"next_day_LSTM_{ticker_symbol}.csv")
    out.to_csv(path, index=False)
    print(f"Saved → {path}")

if __name__ == "__main__":
    main()


Ticker: AAPL, Start: 2015-01-01, End: 2025-03-30


c:\Users\Hamza\anaconda3\envs\myenv\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/25
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - loss: 0.0184 - val_loss: 8.3682e-04
Epoch 2/25
57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 0.0011 - val_loss: 0.0010
Epoch 3/25
57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 0.0011 - val_loss: 7.5006e-04
Epoch 4/25
57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 0.0010 - val_loss: 7.6129e-04
Epoch 5/25
57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 0.0010 - val_loss: 6.9371e-04
Epoch 6/25
57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 9.8656e-04 - val_loss: 7.2290e-04
Epoch 7/25
57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 8.8192e-04 - val_loss: 0.0010
Epoch 8/25
57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 9.7501e-04 - val_loss: 8.7897e-04
Epoch 9/25
57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 7.3487e-04 - val_loss: 6.9048e-04
Epoch 10/25
57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - loss: 6.9585e-04 - val_loss: 7.3414e-04
Epoch 11/25
57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 7.3598e-04 - val_loss: 6.7301e-04
Epo

NameError: name '__file__' is not defined